In [1]:
import os
import json
import requests
import base64
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime, timedelta

In [2]:
def call_refresh_api(refreshtoken: str):
    """
    idTokenをリフレッシュするメソッド。

    Parameters
    ----------
    refreshtoken : str
        refreshtoken。ログイン後の画面からご確認いただけます。

    Returns
    -------
    resjson : dict
        新しいidtokenが格納されたAPIレスポンス(json形式)
    """
    headers = {"accept": "application/json"}
    data = {"refresh-token": refreshtoken}

    response = requests.post(
        "https://api.jpx-jquants.com/refresh", headers=headers, data=json.dumps(data)
    )

    resjson = json.loads(response.text)
    return resjson

In [ ]:
refreshtoken = <refreshtokenを代入する>
call_refresh_api(refreshtoken)

In [4]:
def call_jquants_api(params: dict, idtoken: str, apitype: str, code: str = None):
    """
    J-QuantsのAPIを試すメソッド。

    Parameters
    ----------
    params : dict
        リクエストパラメータ。
    idtoken : str
        idTokenはログイン後の画面からご確認いただけます。
    apitype: str
        APIの種類。"news", "prices", "lists"などがあります。
    code: str
        銘柄を指定するAPIの場合に設定します。

    Returns
    -------
    resjson : dict
        APIレスポンス(json形式)
    """
    datefrom = params.get("datefrom", None)
    dateto = params.get("dateto", None)
    date = params.get("date", None)
    includedetails = params.get("includedetails", "false")
    keyword = params.get("keyword", None)
    headline = params.get("headline", None)
    paramcode = params.get("code", None)
    nexttoken = params.get("nextToken", None)
    headers = {"accept": "application/json", "Authorization": idtoken}
    data = {
        "from": datefrom,
        "to": dateto,
        "includeDetails": includedetails,
        "nextToken": nexttoken,
        "date": date,
        "keyword": keyword,
        "headline": headline,
        "code": paramcode,
    }

    if code:
        code = "/" + code
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype + code,
            params=data,
            headers=headers,
        )
    else:
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype, params=data, headers=headers
        )
    resjson = json.loads(r.text)
    return resjson

In [5]:
idtk=<idtkを代入する>

In [6]:
paramdict = {}
paramdict["includedetails"] = "True"
paramdict["datefrom"] = "2021-01-01"
paramdict["dateto"] = "2021-04-30"

res = call_jquants_api(paramdict, idtk, "lists")
old_df = pd.read_csv("../data/stock_list.csv.gz")
new_df = pd.DataFrame(res["list"])
pd.concat([old_df, new_df]).to_csv("../data/stock_list_latest.csv")

In [7]:
paramdict = {}
paramdict["includedetails"] = "True"
paramdict["datefrom"] = "2021-01-01"
paramdict["dateto"] = "2021-04-30"

res = call_jquants_api(paramdict, idtk, "tdnet")
old_df = pd.read_csv("../data/tdnet.csv.gz")
new_df = pd.DataFrame(res["tdnet"])
pd.concat([old_df, new_df]).to_csv("../data/tdnet_latest.csv")

In [8]:
paramdict = {}
paramdict["includedetails"] = "True"
date_list = pd.date_range(start="2021-01-04", end="2021-04-30", freq="D").to_series().dt.strftime("%Y-%m-%d")

new_df_buff = []
for d in tqdm(date_list):
    paramdict["date"] = d
    res = call_jquants_api(paramdict, idtk, "prices")
    new_df_buff.append(pd.DataFrame(res["prices"]))

old_df = pd.read_csv("../data/stock_price.csv.gz")
new_df = pd.concat(new_df_buff)
pd.concat([old_df, new_df]).to_csv("../data/stock_price_latest.csv")

In [9]:
paramdict = {}
paramdict["includedetails"] = "True"
date_list = pd.date_range(start="2021-01-04", end="2021-04-30", freq="D").to_series().dt.strftime("%Y-%m-%d")

new_df_buff = []
for d in tqdm(date_list):
    paramdict["date"] = d
    res = call_jquants_api(paramdict, idtk, "stockfins")
    new_df_buff.append(pd.DataFrame(res["stockfin"]))

old_df = pd.read_csv("../data/stock_fin.csv.gz")
new_df = pd.concat(new_df_buff)
pd.concat([old_df, new_df]).to_csv("../data/stock_fin_latest.csv")

In [10]:
stock_df = pd.read_csv("../data/stock_price_latest.csv")

In [11]:
stock_df.head()

,Unnamed: 0,Local Code,EndOfDayQuote Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote Volume,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote PreviousClose,EndOfDayQuote PreviousCloseDate,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote VWAP
0,0,1301,2016/01/04,2800.0,2820.0,2740.0,2750.0,2750.0,32000.0,0.1,2770.0,2015/12/30,2770.0,2015/12/30,-20.0,-0.722,2778.250
1,1,1301,2016/01/05,2750.0,2780.0,2750.0,2760.0,2760.0,20100.0,0.1,2750.0,2016/01/04,2750.0,2016/01/04,10.0,0.364,2761.990
2,2,1301,2016/01/06,2760.0,2770.0,2740.0,2760.0,2760.0,15000.0,0.1,2760.0,2016/01/05,2760.0,2016/01/05,0.0,0.000,2758.867
3,3,1301,2016/01/07,2740.0,2760.0,2710.0,2710.0,2710.0,31400.0,0.1,2760.0,2016/01/06,2760.0,2016/01/06,-50.0,-1.812,2733.471
4,4,1301,2016/01/08,2700.0,2740.0,2690.0,2700.0,2700.0,26200.0,0.1,2710.0,2016/01/07,2710.0,2016/01/07,-10.0,-0.369,2709.122


In [14]:
stock_df["EndOfDayQuote Date"].unique()[-200:]

array(['2020/07/08', '2020/07/09', '2020/07/10', '2020/07/13',
       '2020/07/14', '2020/07/15', '2020/07/16', '2020/07/17',
       '2020/07/20', '2020/07/21', '2020/07/22', '2020/07/27',
       '2020/07/28', '2020/07/29', '2020/07/30', '2020/07/31',
       '2020/08/03', '2020/08/04', '2020/08/05', '2020/08/06',
       '2020/08/07', '2020/08/11', '2020/08/12', '2020/08/13',
       '2020/08/14', '2020/08/17', '2020/08/18', '2020/08/19',
       '2020/08/20', '2020/08/21', '2020/08/24', '2020/08/25',
       '2020/08/26', '2020/08/27', '2020/08/28', '2020/08/31',
       '2020/09/01', '2020/09/02', '2020/09/03', '2020/09/04',
       '2020/09/07', '2020/09/08', '2020/09/09', '2020/09/10',
       '2020/09/11', '2020/09/14', '2020/09/15', '2020/09/16',
       '2020/09/17', '2020/09/18', '2020/09/23', '2020/09/24',
       '2020/09/25', '2020/09/28', '2020/09/29', '2020/09/30',
       '2020/10/01', '2020/10/02', '2020/10/05', '2020/10/06',
       '2020/10/07', '2020/10/08', '2020/10/09', '2020/